In [ ]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 20.8 MB/s eta 0:00:00


In [ ]:
import os
import json
import yt_dlp
import cv2
import concurrent.futures
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint


In [ ]:

from google.colab import drive
drive.mount('/content/drive',force_remount=True)

with open('/content/drive/My Drive/Colab Notebooks/MS-ASL/MSASL_train.json', 'r') as file:
    train_data = json.load(file)

with open('/content/drive/My Drive/Colab Notebooks/MS-ASL/MSASL_val.json', 'r') as file:
    val_data = json.load(file)


Mounted at /content/drive


In [ ]:
def download_and_extract_frames(video_data, save_frames_path):
    video_url = video_data['url']
    video_id = video_data['file']
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]',
        'quiet': True,
        'no_warnings': True
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            video_info = ydl.extract_info(video_url, download=False)
            video_url = video_info['url']
    except Exception as e:
        print(f"Error downloading {video_url}: {e}")
        return

    cap = cv2.VideoCapture(video_url)
    output_dir = os.path.join(save_frames_path, video_id)
    os.makedirs(output_dir, exist_ok=True)

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_filename = os.path.join(output_dir, f"frame_{frame_idx}.jpg")
        cv2.imwrite(frame_filename, frame)
        frame_idx += 1
    cap.release()



#os.makedirs(os.path.dirname("/content/drive/My Drive/Colab Notebooks/MS-ASL/preprocessing_checkpoint.txt"), exist_ok=True)

checkpoint_path = "/content/drive/My Drive/Colab Notebooks/MS-ASL/preprocessing_checkpoint.txt"
print(os.path.exists(checkpoint_path))
print(os.listdir("/content/drive/My Drive/Colab Notebooks/MS-ASL/"))

def preprocess_data_multithreaded(data, save_frames_path, max_workers=10, checkpoint_path="/content/drive/My Drive/Colab Notebooks/MS-ASL/preprocessing_checkpoint.txt"):
    # Load existing checkpoints if available
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'r') as file:
            processed_videos = set(line.strip() for line in file.readlines())
    else:
        processed_videos = set()

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Filter out already processed videos and videos without a 'file' key
        filtered_data = [item for item in data if 'file' in item and item['file'] not in processed_videos]

        # Process videos and update checkpoint after each video
        for item in filtered_data:
            download_and_extract_frames(item, save_frames_path)

            # Debugging information
            print(f"Processed video: {item['file']}")

            os.makedirs(os.path.dirname(checkpoint_path), exist_ok=True)
            with open(checkpoint_path, 'a') as file:
                file.write(item['file'] + '\n')

            # Debugging information
            print(f"Updated checkpoint with video: {item['file']}")




True
['C-UDA-0.1_annotated_discussion.pdf', 'MSASL_synonym.json', 'MSASL_classes.json', 'MSASL_test.json', 'MSASL_train.json', 'README.md', 'MSASL_val.json', 'asl_dataset', 'preprocessing_checkpoint.txt']


In [ ]:
# Define the paths where frames will be saved
TRAIN_FRAMES_PATH = "/content/drive/My Drive/Colab Notebooks/MS-ASL/asl_dataset/"
VAL_FRAMES_PATH = "/content/drive/My Drive/Colab Notebooks/MS-ASL/asl_val_dataset/"

# Preprocess the training data
preprocess_data_multithreaded(train_data, TRAIN_FRAMES_PATH)

# Preprocess the validation data
preprocess_data_multithreaded(val_data, VAL_FRAMES_PATH)


Processed video: ASL Boring 
Updated checkpoint with video: ASL Boring 
Processed video: ASL Breakfast  
Updated checkpoint with video: ASL Breakfast  
Processed video: ASL Married 
Updated checkpoint with video: ASL Married 
Processed video: ASL Spider  
Updated checkpoint with video: ASL Spider  
Processed video: ASL I Love You   
Updated checkpoint with video: ASL I Love You   
Processed video: ASL Meat 
Updated checkpoint with video: ASL Meat 
Processed video: ASL English 
Updated checkpoint with video: ASL English 
Processed video: ASL Tiger  
Updated checkpoint with video: ASL Tiger  
Processed video: ASL Friday 
Updated checkpoint with video: ASL Friday 
Processed video: ASL I understand  
Updated checkpoint with video: ASL I understand  
Processed video: ASL Basketball 
Updated checkpoint with video: ASL Basketball 
Processed video: ASL Excited 
Updated checkpoint with video: ASL Excited 
Processed video: ASL Near 
Updated checkpoint with video: ASL Near 
Processed video: ASL F

ERROR: [youtube] W4wQc8awMJo: Video unavailable


Error downloading www.youtube.com/watch?v=W4wQc8awMJo: ERROR: [youtube] W4wQc8awMJo: Video unavailable
Processed video: Baby Sign Language Fifteen
Updated checkpoint with video: Baby Sign Language Fifteen
Processed video: Gray
Updated checkpoint with video: Gray
Processed video: Fruits
Updated checkpoint with video: Fruits
Processed video: Fruits
Updated checkpoint with video: Fruits
Processed video: Fruits
Updated checkpoint with video: Fruits
Processed video: Fruits
Updated checkpoint with video: Fruits
Processed video: Fruits
Updated checkpoint with video: Fruits
Processed video: Fruits
Updated checkpoint with video: Fruits
Processed video: Fruits
Updated checkpoint with video: Fruits
Processed video: Fruits
Updated checkpoint with video: Fruits
Processed video: Fruits
Updated checkpoint with video: Fruits
Processed video: Fruits
Updated checkpoint with video: Fruits
Processed video: Fruits
Updated checkpoint with video: Fruits
Processed video: Fruits
Updated checkpoint with video: 

ERROR: [youtube] YfZNZlekeo4: Video unavailable


Error downloading www.youtube.com/watch?v=YfZNZlekeo4: ERROR: [youtube] YfZNZlekeo4: Video unavailable
Processed video: good morning 2
Updated checkpoint with video: good morning 2
Processed video: rock - ASL sign for rock
Updated checkpoint with video: rock - ASL sign for rock
Processed video: rock - ASL sign for rock
Updated checkpoint with video: rock - ASL sign for rock
Processed video: TEST (version 1)
Updated checkpoint with video: TEST (version 1)


...

Error downloading https://www.youtube.com/watch?v=TUQUVlMupcs: ERROR: [youtube] TUQUVlMupcs: Video unavailable
Processed video: Parameters
Updated checkpoint with video: Parameters


ERROR: [youtube] TUQUVlMupcs: Video unavailable


Error downloading https://www.youtube.com/watch?v=TUQUVlMupcs: ERROR: [youtube] TUQUVlMupcs: Video unavailable
Processed video: Parameters
Updated checkpoint with video: Parameters


ERROR: [youtube] TUQUVlMupcs: Video unavailable


Error downloading https://www.youtube.com/watch?v=TUQUVlMupcs: ERROR: [youtube] TUQUVlMupcs: Video unavailable
Processed video: Parameters
Updated checkpoint with video: Parameters
Processed video: UNIT 9 VOCABULARY
Updated checkpoint with video: UNIT 9 VOCABULARY
Processed video: UNIT 9 VOCABULARY
Updated checkpoint with video: UNIT 9 VOCABULARY
Processed video: UNIT 9 VOCABULARY
Updated checkpoint with video: UNIT 9 VOCABULARY


In [ ]:
def load_data(data_path, input_shape=(224, 224), batch_size=32):
    datagen = ImageDataGenerator(rescale=1./255)
    data_gen = datagen.flow_from_directory(
        data_path,
        target_size=input_shape,
        color_mode="rgb",
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True
    )
    return data_gen


In [ ]:
BATCH_SIZE = 32
EPOCHS = 10

num_classes = len(train_data)
sequence_length = 5

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

model = tf.keras.Sequential([
    TimeDistributed(base_model, input_shape=(sequence_length, 224, 224, 3)),
    TimeDistributed(Flatten()),
    LSTM(256, return_sequences=False, dropout=0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
TRAIN_FRAMES_PATH = "/content/drive/My Drive/Colab Notebooks/MS-ASL/asl_dataset/"
VAL_FRAMES_PATH = "/content/drive/My Drive/Colab Notebooks/MS-ASL/asl_val_dataset/"

train_gen = load_data(TRAIN_FRAMES_PATH, batch_size=BATCH_SIZE)
val_gen = load_data(VAL_FRAMES_PATH, batch_size=BATCH_SIZE)

checkpoint_filepath = "/content/drive/My Drive/Colab Notebooks/MS-ASL/asl_cnn_lstm_checkpoint.h5"
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

# Uncomment the following line when you're ready to train the model
# model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[model_checkpoint_callback])

model_save_path = "/content/drive/My Drive/Colab Notebooks/Saved-Model/asl_cnn_lstm_model.h5"
model.save(model_save_path)
